# Stock Data Pipeline


## 0. Install Zipline

You will need to restart your runtime after installing since zipline uses an older version of pandas.

In [1]:
!pip3 install zipline==1.3.0

     |████████████████████████████████| 2.5MB 5.8MB/s 
     |████████████████████████████████| 92kB 9.4MB/s 
     |████████████████████████████████| 26.3MB 1.7MB/s 
     |████████████████████████████████| 143kB 57.6MB/s 
     |████████████████████████████████| 1.3MB 57.2MB/s 
     |████████████████████████████████| 624kB 41.4MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 163kB 35.9MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 112kB 46.1MB/s 
  Created wheel for zipline: filename=zipline-1.3.0-cp36-cp36m-linux_x86_64.whl size=5008415 sha256=44a9c792c34d40b86bc7054243b41923463d902c5b2d3a03295fdb39298b939d
  Stored in directory: /root/.cache/pip/wheels/a4/d6/67/f303ab028b004bf8e00c05b5b04fba83d8ec238b6547becdb7
  Created wheel for Logbook: filename=Logbook-1.5.3-cp36-cp36m-linux_x86_64.whl size=66381 sha256=fe624c4025ecf8d35de2f7e5f5e03af247850438be7cea909bbf5ff54161b627
  Stored in 

## 1. Load libraries data directories

In [1]:
import zipline

from collections import OrderedDict
import numpy as np

import pandas as pd
from os import listdir
import sys

import os

from zipline.data import bundles
from zipline.pipeline import Pipeline
from zipline.utils.calendars import get_calendar
from zipline.pipeline.engine import SimplePipelineEngine
from zipline.pipeline.factors import CustomFactor, DailyReturns, AverageDollarVolume


from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.loaders import USEquityPricingLoader
from zipline.assets._assets import Equity
from zipline.api import symbol

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Zipline root directory
zipline_dir = '/content/drive/MyDrive/abnormal-distribution-project-data/zipline'

os.environ['ZIPLINE_ROOT'] = zipline_dir

## 2. Ingestion and raw data processing 

### 2.1 Ingestion and Data Processing functions

In [93]:
# Portions of this code adapted from https://github.com/pbharrin/alpha-compiler


METADATA_HEADERS = ['start_date', 'end_date', 'auto_close_date',
                    'symbol', 'exchange', 'asset_name']


def check_for_abnormal_returns(df, thresh=3.0):
    """Checks to see if any days have abnormal returns"""
    returns = df['close'].pct_change()
    abnormal_rets = returns[returns > thresh]
    if abnormal_rets.shape[0] > 0:
        sys.stderr.write('Abnormal returns for: {}\n'.format(df.ix[0]['ticker']))
        sys.stderr.write('{}\n'.format(str(abnormal_rets)))


def from_sep_dump(file_name, start=None, end=None):
    """
    Function that reads full Sharadar stock price csv file
    sharadar_sep.csv, and returns an ingest function for Zipline

    """
    us_calendar = get_calendar("NYSE").all_sessions
    ticker2sid_map = {}

    def ingest(environ,
               asset_db_writer,
               minute_bar_writer,  # unused
               daily_bar_writer,
               adjustment_writer,
               calendar,
               cache,
               show_progress,
               output_dir,
               # pass these as defaults to make them 'nonlocal' in py2
               start=start,
               end=end):

        print("starting ingesting data from: {}".format(file_name))

        # read in the whole dump (will require ~7GB of RAM)
        df = pd.read_csv(file_name, index_col='date',
                         parse_dates=['date'], na_values=['NA'])

        # drop unused columns, dividends will be used later
        df = df.drop(['lastupdated', 'dividends', 'closeunadj'], axis=1)

        # counter of valid securites, this will be our primary key
        sec_counter = 0
        data_list = []  # list to send to daily_bar_writer
        metadata_list = []  # list to send to asset_db_writer (metadata)

        # iterate over all the unique securities and pack data, and metadata
        # for writing
        for tkr, df_tkr in df.groupby('ticker'):
            df_tkr = df_tkr.sort_index()

            row0 = df_tkr.ix[0]  # get metadata from row

            print(" preparing {}".format(row0["ticker"]))
            check_for_abnormal_returns(df_tkr)

            # check to see if there are missing dates in the middle
            this_cal = us_calendar[(us_calendar >= df_tkr.index[0]) & (us_calendar <= df_tkr.index[-1])]
            if len(this_cal) != df_tkr.shape[0]:
                print('MISSING interstitial dates for: %s using forward fill' % row0["ticker"])
                print('number of dates missing: {}'.format(len(this_cal) - df_tkr.shape[0]))
                df_desired = pd.DataFrame(index=this_cal.tz_localize(None))
                df_desired = df_desired.join(df_tkr)
                df_tkr = df_desired.fillna(method='ffill')

            # update metadata; 'start_date', 'end_date', 'auto_close_date',
            # 'symbol', 'exchange', 'asset_name'
            metadata_list.append((df_tkr.index[0],
                                  df_tkr.index[-1],
                                  df_tkr.index[-1] + pd.Timedelta(days=1),
                                  row0["ticker"],
                                  "SEP",  # all have exchange = SEP
                                  row0["ticker"]  # TODO: can we delete this?
                                  )
                                 )

            # drop metadata columns
            df_tkr = df_tkr.drop(['ticker'], axis=1)

            # pack data to be written by daily_bar_writer
            data_list.append((sec_counter, df_tkr))
            ticker2sid_map[tkr] = sec_counter  # record the sid for use later
            sec_counter += 1

        print("writing data for {} securities".format(len(metadata_list)))
        daily_bar_writer.write(data_list, show_progress=False)

        # write metadata
        asset_db_writer.write(equities=pd.DataFrame(metadata_list,
                                                    columns=METADATA_HEADERS))
        print("a total of {} securities were loaded into this bundle".format(
            sec_counter))

        # read in Dividend History
        dfd = pd.read_csv(file_name, index_col='date',
                         parse_dates=['date'], na_values=['NA'])
        # drop rows where dividends == 0.0
        dfd = dfd[dfd["dividends"] != 0.0]
        dfd = dfd.dropna()

        dfd.loc[:, 'ex_date'] = dfd.loc[:, 'record_date'] = dfd.index
        dfd.loc[:, 'declared_date'] = dfd.loc[:, 'pay_date'] = dfd.index
        dfd.loc[:, 'sid'] = dfd.loc[:, 'ticker'].apply(lambda x: ticker2sid_map[x])
        dfd = dfd.rename(columns={'dividends': 'amount'})
        dfd = dfd.drop(['open', 'high', 'low', 'close', 'volume', 'lastupdated', 'ticker', 'closeunadj'], axis=1)

        # # format dfd to have sid
        adjustment_writer.write(dividends=dfd)

    return ingest

def get_tickers_from_bundle(bundle_name):
    """Gets a list of tickers from a given bundle"""
    bundle_data = bundles.load(bundle_name, os.environ, None)

    # get a list of all sids
    lifetimes = bundle_data.asset_finder._compute_asset_lifetimes()
    all_sids = lifetimes.sid

    # retreive all assets in the bundle
    all_assets = bundle_data.asset_finder.retrieve_all(all_sids)

    # return only tickers
    return map(lambda x: (x.symbol, x.sid), all_assets)


def get_all_assets_for_bundle(bundle_name):
    """For a given bundle get a list of all assets"""
    bundle_data = load(bundle_name, os.environ, None)

    # get a list of all sids
    lifetimes = bundle_data.asset_finder._compute_asset_lifetimes()
    all_sids = lifetimes.sid

    print('all_sids: ', all_sids)

    # retreive all assets in the bundle
    return bundle_data.asset_finder.retrieve_all(sids=all_sids)


def get_ticker_sid_dict_from_bundle(bundle_name):
    """Packs the (ticker,sid) tuples into a dict."""
    all_equities = get_tickers_from_bundle(bundle_name)
    return dict(all_equities)

def pack_sparse_data(N, rawpath, fields, filename):
    """pack data into np.recarray and persists it to a file to be
    used by SparseDataFactor"""


    # create buffer to hold data for all tickers
    dfs = [None] * N

    max_len = -1
    print("Packing sids")
    for fn in listdir(rawpath):
        if not fn.endswith(".csv"):
            continue
        df = pd.read_csv(os.path.join(rawpath,fn), index_col="Date", parse_dates=True)
        df = df.sort_index()
        sid = int(fn.split('.')[0])
        #print("packing sid: %d" % sid)
        dfs[sid] = df

        # width is max number of rows in any file
        max_len = max(max_len, df.shape[0])
    print("Finished packing sids")

    # temp workaround for `Array Index Out of Bound` bug
    max_len = max_len + 1

    # pack up data as buffer
    num_fundamentals = len(fields)
    buff = np.full((num_fundamentals + 1, N, max_len), np.nan)

    dtypes = [('date', '<f8')]
    for field in fields:
        dtypes.append((field, '<f8'))

    # pack self.data as np.recarray
    data = np.recarray(shape=(N, max_len), buf=buff, dtype=dtypes)

    # iterate over loaded data and populate self.data
    for i, df in enumerate(dfs):
        if df is None:
            continue
        ind_len = df.index.shape[0]
        data.date[i, :ind_len] = df.index
        for field in fields:
            data[field][i, :ind_len] = df[field]

    data.dump(filename)  # can be read back with np.load()


def load_sf1(sf1_dir, 
             fields, 
             bund ='sep', 
             npy_name='SF1', 
             stocks_dir = '/content/drive/MyDrive/abnormal-distribution-project-data/stocks/dummy',
             dimensions=None):
    """
    Loads SF1 data into a npy compressed file SF1.npy
    :param sf1_dir: Sharadar SF1 bulk file
    :param fields: fields to load
    :param dimensions: dimensions to load. One-to-one with fields. If None, assume ARQ if data available,
    ART if not
    """

    bundles.register(bund, from_sep_dump('.', '.'), )
    num_tickers = len(get_ticker_sid_dict_from_bundle(bund))
    print('number of tickers: ', num_tickers)

    data = pd.read_csv(sf1_dir)

    tickers = get_ticker_sid_dict_from_bundle(bund)
    
    counter = 0
    for ticker, sid in tickers.items():
        counter += 1
        if counter % 100 == 0:
            print("Working on {}-th file".format(counter))

        df = data[(data.ticker == ticker)]
        df = df.rename(columns={'datekey': 'Date'}).set_index('Date')
        df.index = df.index.rename('Date')
        series = []
        for i, field in enumerate(fields):
            if dimensions is None:
                if df[df.dimension == 'ARQ'][field].isna().sum() == df[df.dimension == 'ARQ'].shape[0]:
                    s = df[df.dimension == 'ART'][field]
                else:
                    s = df[df.dimension == 'ARQ'][field]
            else:
                s = df[df.dimension == dimensions[i]][field]
            series.append(s)

        df = pd.concat(series, axis=1)
        df = df.sort_index()
        df.index = df.index.rename('Date')
        df.to_csv(os.path.join(stocks_dir, "{}.csv".format(sid)))
    
    pack_sparse_data(num_tickers + 1,  # number of tickers in bundle + 1
                     stocks_dir,
                     fields,
                     zipline_dir + '/data/' + npy_name +  '.npy')  # write directly to the zipline data dir


class SparseDataFactor(CustomFactor):
    """Abstract Base Class to be used for computing sparse data.
    The data is packed and persisted into a NumPy binary data file
    in a previous step.
    This class must be subclassed with class variable 'outputs' set.  The fields
    in 'outputs' should match those persisted."""
    inputs = []
    window_length = 1

    def __init__(self, *args, **kwargs):
        self.time_index = None
        self.curr_date = None # date for which time_index is accurate
        self.data = None
        self.data_path = "please_specify_.npy_file"

    def bs(self, arr):
        """Binary Search"""
        if len(arr) == 1:
            if self.curr_date < arr[0]:
                return 0
            else: return 1

        mid = int(len(arr) / 2)
        if self.curr_date < arr[mid]:
            return self.bs(arr[:mid])
        else:
            return mid + self.bs(arr[mid:])

    def bs_sparse_time(self, sid):
        """For each security find the best range in the sparse data."""
        dates_for_sid = self.data.date[sid]
        if np.isnan(dates_for_sid[0]):
            return 0

        # do a binary search of the dates array finding the index
        # where self.curr_date will lie.
        non_nan_dates = dates_for_sid[~np.isnan(dates_for_sid)]
        return self.bs(non_nan_dates) - 1

    def cold_start(self, today, assets):
        if self.data is None:
            self.data = np.load(self.data_path, allow_pickle=True)

        self.M = self.data.date.shape[1]

        # for each sid, do binary search of date array to find current index
        # the results can be shared across all factors that inherit from SparseDataFactor
        # this sets an array of ints: time_index
        self.time_index = np.full(self.N, -1, np.dtype('int64'))
        self.curr_date = today.value
        for asset in assets:  # asset is numpy.int64
            self.time_index[asset] = self.bs_sparse_time(asset)

    def update_time_index(self, today, assets):
        """Ratchet update.
        for each asset check if today >= dates[self.time_index]
        if so then increment self.time_index[asset.sid] += 1"""

        ind_p1 = self.time_index.copy()
        np.add.at(ind_p1, ind_p1 != (self.M - 1), 1)
        sids_to_increment = today.value >= self.data.date[np.arange(self.N), ind_p1]
        sids_not_max = self.time_index != (self.M - 1)   # create mask of non-maxed
        self.time_index[sids_to_increment & sids_not_max] += 1

        self.curr_date = today.value

    def compute(self, today, assets, out, *arrays):
        # for each asset in assets determine index from date (today)
        if self.time_index is None:
            self.cold_start(today, assets)
        else:
            self.update_time_index(today, assets)

        ti_used_today = self.time_index[assets]

        for field in self.__class__.outputs:
            out[field][:] = self.data[field][assets, ti_used_today]


class Fundamentals(SparseDataFactor):
    outputs = ['marketcap', 'assets', 'liabilities', 'pe', 'currentratio', 'netmargin', 'capex', 'fcf', 'roic']

    def __init__(self, *args, **kwargs):
        super(Fundamentals, self).__init__(*args, **kwargs)
        self.N = len(get_ticker_sid_dict_from_bundle("sep")) + 1  # max(sid)+1 get this from the bundle
        self.data_path = zipline_dir + '/data/' + 'SF1.npy'

class FundamentalsSP500(SparseDataFactor):
    outputs = ['marketcap', 'assets', 'liabilities', 'pe', 'currentratio', 'netmargin', 'capex', 'fcf', 'roic']

    def __init__(self, *args, **kwargs):
        super(FundamentalsSP500, self).__init__(*args, **kwargs)
        self.N = len(get_ticker_sid_dict_from_bundle("sp500")) + 1  # max(sid)+1 get this from the bundle
        self.data_path = zipline_dir + '/data/' + 'SF1_SP500.npy'

class FundamentalsSP1500(SparseDataFactor):
    outputs = ['marketcap', 'assets', 'liabilities', 'pe', 'currentratio', 'netmargin', 'capex', 'fcf', 'roic']

    def __init__(self, *args, **kwargs):
        super(FundamentalsSP1500, self).__init__(*args, **kwargs)
        self.N = len(get_ticker_sid_dict_from_bundle("sp1500")) + 1  # max(sid)+1 get this from the bundle
        self.data_path = zipline_dir + '/data/' + 'SF1.npy'

### 2.2 Ingest stock and fundamental data into Zipine

Warning: Running this code will overwrite database and can take almost an hour to run!

In [ ]:
# Stock price data ingestion

#!zipline ingest -b 'sep'
#zipline ingest -b 'sp500'
#!zipline ingest -b 'sp1500'

In [ ]:
# Fundamental data ingestion

# Choose which fundamental fields you want to be able to process through the pipeline.
# A list of  of all available fields can be found below.

# Fundamental data directory
#sf1_dir = '/content/drive/MyDrive/abnormal-distribution-project-data/stocks/SHARADAR_SF1.csv'
#fields = ['marketcap', 'assets', 'liabilities', 'pe', 'currentratio', 'netmargin', 'capex', 'fcf', 'roic']
#load_sf1(sf1_dir, fields, dimensions=None)


An explanation of database related fields can be found in :

  https://www.quandl.com/databases/SF1/documentation?anchor=dimensions

A summary of each fundamental field can be found in:

https://docs-1-8--quantrocket.netlify.app/docs/data/fundamental/sharadar/
        
        ['ticker', 'dimension', 'calendardate', 'datekey', 'reportperiod',
       'lastupdated', 'accoci', 'assets', 'assetsavg', 'assetsc', 'assetsnc',
       'assetturnover', 'bvps', 'capex', 'cashneq', 'cashnequsd', 'cor',
       'consolinc', 'currentratio', 'de', 'debt', 'debtc', 'debtnc', 'debtusd',
       'deferredrev', 'depamor', 'deposits', 'divyield', 'dps', 'ebit',
       'ebitda', 'ebitdamargin', 'ebitdausd', 'ebitusd', 'ebt', 'eps',
       'epsdil', 'epsusd', 'equity', 'equityavg', 'equityusd', 'ev', 'evebit',
       'evebitda', 'fcf', 'fcfps', 'fxusd', 'gp', 'grossmargin']

       ['intexp', 'invcap', 'invcapavg', 'inventory', 'investments',
       'investmentsc', 'investmentsnc', 'liabilities', 'liabilitiesc',
       'liabilitiesnc', 'marketcap', 'ncf', 'ncfbus', 'ncfcommon', 'ncfdebt',
       'ncfdiv', 'ncff', 'ncfi', 'ncfinv', 'ncfo', 'ncfx', 'netinc',
       'netinccmn', 'netinccmnusd', 'netincdis', 'netincnci', 'netmargin',
       'opex', 'opinc', 'payables', 'payoutratio', 'pb', 'pe', 'pe1',
       'ppnenet', 'prefdivis', 'price', 'ps', 'ps1', 'receivables', 'retearn',
       'revenue', 'revenueusd', 'rnd', 'roa', 'roe', 'roic', 'ros', 'sbcomp',
       'sgna', 'sharefactor', 'sharesbas', 'shareswa', 'shareswadil', 'sps',
       'tangibles', 'taxassets', 'taxexp', 'taxliabilities', 'tbvps',
       'workingcapital']

# 3. Equity factors

## 3.1 Load helper functions

In [5]:
# Portions of these code adapted from https://www.udacity.com/course/ai-for-trading--nd880 


def register_data(start_date, end_date, bundle_name, address):

    start_session = pd.Timestamp(start_date, tz='utc')
    end_session = pd.Timestamp(end_date, tz='utc')

    register(bundle_name, csvdir_equities(['daily'],address,),
    calendar_name='NYSE', start_session=start_session,
    end_session=end_session)


class PricingLoader(object):
    def __init__(self, bundle_data):
        self.loader = USEquityPricingLoader(
            bundle_data.equity_daily_bar_reader,
            bundle_data.adjustment_reader)

    def get_loader(self, column):
        if column not in USEquityPricing.columns:
            raise Exception('Column not in USEquityPricing')
        return self.loader

def build_pipeline_engine(bundle_data, trading_calendar):
    pricing_loader = PricingLoader(bundle_data)

    engine = SimplePipelineEngine(
        get_loader=pricing_loader.get_loader,
        calendar=trading_calendar.all_sessions,
        asset_finder=bundle_data.asset_finder)

    return engine

# Loading stock list from file
def stock_list(file_name):
    all_stocks = []
    with open(file_name, 'r') as f:
        for line in f:
            # remove linebreak which is the last character of the string
            currentPlace = line[:-1]
            # add item to the list
            all_stocks.append(currentPlace)
        return all_stocks

def get_universe_tickers(engine, universe, end_date):
    universe_end_date = pd.Timestamp(end_date, tz='UTC')

    universe_tickers = engine \
        .run_pipeline(
        Pipeline(screen=universe),
        universe_end_date,
        universe_end_date) \
        .index.get_level_values(1) \
        .values.tolist()

    return universe_tickers


def run_pipeline(engine, pipeline, start_date, end_date):

    # TODO: adjust for trading days
    end_dt = pd.Timestamp(end_date.strftime('%Y-%m-%d'), tz='UTC')
    start_dt = pd.Timestamp(start_date.strftime('%Y-%m-%d'), tz='UTC')
    return engine.run_pipeline(pipeline, start_dt, end_dt)


def get_pipeline_tickers(factors):

    return factors.index.levels[1].values.tolist()


def make_pipeline(factors, universe):
    factors_pipe = OrderedDict()
        
    for name, f in factors.items():
        factors_pipe[name] = f
                    
    pipe = Pipeline(screen=universe, columns=factors_pipe)
    
    return pipe

#pd.DataFrame(sorted([asset.security_name for asset in all_assets]), 
#             columns =['ticker']).to_csv('sp1500.csv', index=False)

#data = pd.read_csv('/content/drive/MyDrive/abnormal-distribution-project-data/stocks/SHARADAR_SEP.csv')
#sp500_tickers = pd.read_csv('/content/drive/MyDrive/abnormal-distribution-project-data/components/sp500.csv')
#data_sp500 = data[data.ticker.isin(sp500_tickers.ticker)]
#data_sp500.reset_index(inplace=True, drop=True)
#data_sp500.to_csv('/content/drive/MyDrive/abnormal-distribution-project-data/stocks/SHARADAR_SP500.csv', index=False)

## 3.2 Custom Factors


In [6]:
def make_factors():
    
    all_factors = {
        '1Y_return': DailyReturns(window_length=252)
    }
    
    return all_factors

## 3.3 Run pipeline


In [94]:
# Obtain data, choose trading calendar and build pipeline engine

trading_calendar = get_calendar('NYSE') 
bundles.register('sp1500', from_sep_dump('.'))
bundle_data = bundles.load('sp1500')
engine = build_pipeline_engine(bundle_data, trading_calendar)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Overwriting bundle with name 'sp1500'
  after removing the cwd from sys.path.


In [95]:
# Set date range
start_date = '2009-12-30'
end_date = '2020-11-19'
universe_start_date = pd.Timestamp(start_date, tz='UTC')
universe_end_date = pd.Timestamp(end_date, tz='UTC')

# Select universe of stocks
universe = FundamentalsSP1500().marketcap.top(1500) 

# Define pipeline
pipeline = Pipeline(screen=universe)
pipeline.add(Fundamentals().marketcap, 'universe')
pipeline.add(DailyReturns(window_length=252), '1Y_return')


In [96]:
# Run pipeline 
all_factors = run_pipeline(engine, pipeline, universe_start_date, universe_end_date)

# Get all tickers for the stocks we're looking at
all_assets = get_pipeline_tickers(all_factors)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:314: RuntimeWarning: invalid value encountered in less_equal


In [97]:
all_factors

1Y_return      universe
2009-12-30 00:00:00+00:00 Equity(0 [A])         1.038306  1.043354e+10
                          Equity(2 [AAAGY])     0.228169  1.364417e+08
                          Equity(6 [AAPL])      1.423102  3.304642e+07
                          Equity(8 [AB])        0.558481  1.255418e+09
                          Equity(9 [ABB])       0.324371  1.633537e+08
                          Equity(11 [ABC])      0.507706  1.520995e+08
                          Equity(16 [ACAD])     0.569767  2.170741e+08
                          Equity(19 [ACC])      0.501006  2.188535e+08
                          Equity(20 [ACF])      1.682517  6.216400e+08
                          Equity(21 [ACGL])     0.026874  4.166472e+08
                          Equity(22 [ACH])      1.062074  1.004267e+08
                          Equity(25 [ACIIQ])    0.477184  4.287837e+06
                          Equity(26 [ACL])      0.950880  5.863109e+06
                          Equity(27 [ACM])      0.005357  4.580467e+07
                          Equity(31 [ADBE])     0.767696  2.898130e+07
                          Equity(32 [ADI])      0.718040  2.244076e+09
                          Equity(33 [ADM])      0.121584  1.144910e+09
                          Equity(37 [ADS])      0.405217  3.796605e+09
                          Equity(42 [AEG])      0.009217  1.353188e+08
                          Equity(43 [AEM])      0.086055  7.622483e+07
                          Equity(45 [AEP])      0.163565  3.505489e+07
                          Equity(48 [AET])      0.120713  8.628984e+06
                          Equity(49 [AETUF])    0.316366  1.358384e+08
                          Equity(50 [AFG])      0.137477  5.981298e+08
                          Equity(51 [AFL])      0.065723  2.732912e+08
                          Equity(52 [AFSI])     0.077784  2.445896e+09
                          Equity(54 [AGN])      0.542327  7.360154e+07
                          Equity(55 [AGN1])     0.624362  2.441999e+08
                          Equity(56 [AGNC])     0.606664  5.146541e+08
                          Equity(57 [AGO])      1.104736  4.304135e+10
...                                                  ...           ...
2020-11-19 00:00:00+00:00 Equity(1932 [WU])    -0.167995  4.972810e+05
                          Equity(1934 [WW])    -0.379268  6.628674e+09
                          Equity(1936 [WWD])   -0.029075  2.589034e+08
                          Equity(1937 [WWE])   -0.304446  6.897710e+05
                          Equity(1938 [WY])    -0.001429  7.060098e+06
                          Equity(1939 [WYND])  -0.043065  3.898694e+08
                          Equity(1940 [WYNN])  -0.179504  1.297955e+09
                          Equity(1941 [X])     -0.140695  1.458797e+08
                          Equity(1943 [XEC])   -0.237791  2.078317e+09
                          Equity(1946 [XLNX])   0.440074  1.047823e+10
                          Equity(1947 [XLRN])   1.518672  1.248134e+09
                          Equity(1948 [XOM])   -0.409232  6.932341e+07
                          Equity(1949 [XP])          NaN  4.890043e+08
                          Equity(1950 [XPEV])        NaN  2.077872e+08
                          Equity(1951 [XPO])    0.282675  4.931126e+06
                          Equity(1952 [XRAY])  -0.128263  1.248225e+08
                          Equity(1953 [XRX])   -0.399825  3.056878e+07
                          Equity(1955 [XYL])    0.242239  7.348062e+07
                          Equity(1956 [Y])     -0.215923  6.333728e+09
                          Equity(1957 [YELP])  -0.097575  1.074552e+08
                          Equity(1959 [YNDX])   0.537235  3.924465e+06
                          Equity(1961 [YPF])   -0.484783  1.925349e+08
                          Equity(1963 [YUMC])   0.416801  1.461237e+08
                          Equity(1966 [ZBH])    0.047688  2.584477e+08
                          Equity(1967 [ZBRA])   0.392312  5.828940e+08
